In this notebook we will explore [Hugging Face Transformers](https://huggingface.co/docs/transformers/index).
You may also want to check the [Hugging Face course](https://huggingface.co/course/), which will explain you how to use this technology in a much greater depth.

Training transformer models is computationally expensive. Hugging Face makes available several pretrained [models](https://huggingface.co/models) that can be used as is, or fine-tuned to a specific NLP task, such as one of sentence classification. That's what we'll do in this notebook.

Hugging Face also makes available several [datasets](https://huggingface.co/datasets) that can be used to train or fine-tune a model.

See:
- https://huggingface.co/docs/transformers/tasks/sequence_classification#preprocess
- https://huggingface.co/docs/transformers/training#prepare-a-dataset
- https://huggingface.co/docs/transformers/accelerate
- https://huggingface.co/docs/transformers/model_summary#autoencoding-models

In [ ]:
!pip install pandas
!pip install datasets
!pip install transformers
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

## Preparing our Data

In this notebook, we'll start by using a local dataset (instead of using a dataset stored at Hugging Face).
Let's load data for our classification task.

### Loading dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

# Importing the dataset
dataset = pd.read_excel('/content/drive/Shareddrives/PLN/Assignment 2/data/OpArticles_ADUs.xlsx')
dataset = dataset.drop(columns=['article_id', 'annotator', 'node','ranges'])
dataset['label'].replace(['Value', 'Value(+)', 'Value(-)', 'Fact', 'Policy'],[0,1,2,3,4], inplace=True)

print(dataset.info())
print(dataset.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16743 entries, 0 to 16742
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tokens  16743 non-null  object
 1   label   16743 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 261.7+ KB
None
                                              tokens  label
0           O facto não é apenas fruto da ignorância      0
1  havia no seu humor mais jornalismo (mais inves...      0
2                              É tudo cómico na FIFA      0
3  o que todos nós permitimos que esta organizaçã...      0
4            não nos fazem rir à custa dos poderosos      0


For ease of usage with Transformer models, we convert the dataset into a Hugging Face dataset and split it into train, validation and test sets.

In [4]:
from datasets import Dataset

dataset_hf = Dataset.from_pandas(dataset)

In [5]:
from datasets import DatasetDict

# 90% train, 10% test+validation
train_test = dataset_hf.train_test_split(test_size=0.1, shuffle=True, seed=42)

# Split the 10% test+validation set in half test, half validation
valid_test = train_test['test'].train_test_split(test_size=0.5, shuffle=True, seed=42)

# gather everyone if you want to have a single DatasetDict
train_valid_test_dataset = DatasetDict({
    'train': train_test['train'],
    'validation': valid_test['train'],
    'test': valid_test['test']
})

In [6]:
train_valid_test_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'label'],
        num_rows: 15068
    })
    validation: Dataset({
        features: ['tokens', 'label'],
        num_rows: 837
    })
    test: Dataset({
        features: ['tokens', 'label'],
        num_rows: 838
    })
})

## Fine-tuning a pretrained model

### Tokenizer

We first load the tokenizer for our model:

In [7]:
from transformers import AutoTokenizer

def get_tokenizer(name):
    return AutoTokenizer.from_pretrained(name)

Now we need to [preprocess](https://huggingface.co/docs/transformers/preprocessing) our data.

Obtaining the length of the longest sequences in our data splits

In [8]:
def find_max_length(dataset):
    return len(max(dataset, key=lambda x: len(x.split())).split())

train_max_length = find_max_length(train_valid_test_dataset["train"]["tokens"])
val_max_length = find_max_length(train_valid_test_dataset["validation"]["tokens"])
test_max_length = find_max_length(train_valid_test_dataset["test"]["tokens"])

print(f"Longest sequence in train set has {train_max_length} words")
print(f"Longest sequence in val set has {val_max_length} words")
print(f"Longest sequence in test set has {test_max_length} words")

Longest sequence in train set has 81 words
Longest sequence in val set has 51 words
Longest sequence in test set has 59 words


Tokenize entire dataset

In [9]:
# Define tokenizer later
tokenizer = None

def tokenize_dataset(sample):
    return tokenizer(sample["tokens"], truncation=True, max_length=81, padding="max_length")

def get_tokenized_data(dataset):
    return dataset.map(tokenize_dataset, batched=True)

### Loading the model

Since we want to use the model for classification, we should load it with an appropriate classification head:

In [10]:
from transformers import AutoModelForSequenceClassification
import torch

def get_model(name):
    model = AutoModelForSequenceClassification.from_pretrained(name, num_labels=5)
    model.cuda() # Use GPU

    return model

### Fine-tuning

The next step is to [fine-tune](https://huggingface.co/docs/transformers/training) the model with our train data. To do so, we can make use of a [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer).
There are several aspects of training that you can specify via [TrainingArguments](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments).

In [11]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import numpy as np

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def get_trainingArgs():
    return TrainingArguments(
        output_dir="./results",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3  ,
        weight_decay=0.01,
        data_seed=42,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1"
    )

def get_trainer(model_, args_, dataset_, tokenizer_, data_collator_, compute_metrics_):
    return Trainer(
        model=model_,
        args=args_,
        train_dataset=dataset_["train"],
        eval_dataset=dataset_["validation"],
        tokenizer=tokenizer_,
        data_collator=data_collator_,
        compute_metrics=compute_metrics_
    )

#### Train, evaluate, predict, save

In [12]:
from transformers import DataCollatorWithPadding
from IPython.display import display

def train_model(model_name):
  global tokenizer
  tokenizer = get_tokenizer(model_name)
  tokenized_dataset = get_tokenized_data(train_valid_test_dataset)
  model = get_model(model_name)

  trainer = get_trainer(
    model,
    get_trainingArgs(),
    tokenized_dataset,
    tokenizer,
    DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics
    )
  
  # Train Model
  display(trainer.train())

  # Check performance in validation set
  display(trainer.evaluate())

  # Check how the model fares in our test set.
  display(trainer.predict(test_dataset=tokenized_dataset["test"]))

  # Save model for future use
  trainer.save_model('/content/drive/Shareddrives/PLN/Assignment 2/models/' + model_name)

### PT - neuralmind/bert-base-portuguese-cased

As a starting example, we'll use a lighter BERT-based model. We will need to load:
- the [tokenizer](https://huggingface.co/docs/transformers/autoclass_tutorial#autotokenizer) (which is used to [preprocess](https://huggingface.co/docs/transformers/preprocessing) the data before it can be used by the model)
- the [model](https://huggingface.co/docs/transformers/autoclass_tutorial#automodel) itself

In [16]:
train_model("neuralmind/bert-base-portuguese-cased")

loading configuration file https://huggingface.co/neuralmind/bert-base-portuguese-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e716e2151985ba669e7197b64cdde2552acee146494d40ffaf0688a3f152e6ed.18a0b8b86f3ebd4c8a1d8d6199178feae9971ff5420f1d12f0ed8326ffdff716
Model config BertConfig {
  "_name_or_path": "neuralmind/bert-base-portuguese-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_tran

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/neuralmind/bert-base-portuguese-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e716e2151985ba669e7197b64cdde2552acee146494d40ffaf0688a3f152e6ed.18a0b8b86f3ebd4c8a1d8d6199178feae9971ff5420f1d12f0ed8326ffdff716
Model config BertConfig {
  "_name_or_path": "neuralmind/bert-base-portuguese-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 

...

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 837
  Batch size = 16


{'epoch': 3.0,
 'eval_accuracy': 0.6081242532855436,
 'eval_f1': 0.5897821153208468,
 'eval_loss': 0.9645479321479797,
 'eval_precision': 0.6035410312273057,
 'eval_recall': 0.5929285987017947,
 'eval_runtime': 4.6539,
 'eval_samples_per_second': 179.848,
 'eval_steps_per_second': 11.388}

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 838
  Batch size = 16


PredictionOutput(predictions=array([[ 1.884908  ,  1.038498  , -2.18935   , -1.6066478 , -0.35243088],
       [ 3.4300473 , -1.994894  ,  0.63945967,  0.72143704, -3.1212835 ],
       [ 3.0750031 , -2.2487645 , -0.12322539, -0.03730537, -1.1943377 ],
       ...,
       [ 2.7760584 , -2.4414334 ,  1.3514451 ,  1.5902003 , -3.3672647 ],
       [ 1.2131273 , -2.2941365 ,  3.2757924 ,  0.8694912 , -2.8195364 ],
       [ 2.1537352 , -2.6220095 ,  2.099932  ,  1.64306   , -3.1421173 ]],
      dtype=float32), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/neuralmind/bert-base-portuguese-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/neuralmind/bert-base-portuguese-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/neuralmind/bert-base-portuguese-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/neuralmind/bert-base-portuguese-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/neuralmind/bert-base-portuguese-cased/special_tokens_map.json


In [14]:
!rm -rf ./results/

### PT - "neuralmind/bert-large-portuguese-cased"

In [13]:
train_model("neuralmind/bert-large-portuguese-cased")

Downloading:   0%|          | 0.00/155 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/648 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 837
  Batch size = 16


{'epoch': 3.0,
 'eval_accuracy': 0.6308243727598566,
 'eval_f1': 0.6045392345284912,
 'eval_loss': 0.955284833908081,
 'eval_precision': 0.6210347708739441,
 'eval_recall': 0.6106998309060165,
 'eval_runtime': 14.272,
 'eval_samples_per_second': 58.646,
 'eval_steps_per_second': 3.714}

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 838
  Batch size = 16


PredictionOutput(predictions=array([[ 1.637802  ,  0.7962177 , -2.72708   , -0.68489033,  1.296607  ],
       [ 3.6501598 , -1.5356071 ,  0.48500237,  1.2069862 , -3.5583043 ],
       [ 3.4513097 , -1.6502944 ,  0.52385294,  1.2900612 , -3.3338583 ],
       ...,
       [ 3.1174316 , -2.503461  ,  1.4181416 ,  2.171452  , -3.7076838 ],
       [ 2.0292697 , -2.699613  ,  2.9485495 ,  1.1874743 , -3.2200408 ],
       [ 2.8544135 , -2.4888122 ,  1.6299777 ,  2.1279726 , -4.0976453 ]],
      dtype=float32), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/neuralmind/bert-large-portuguese-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/neuralmind/bert-large-portuguese-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/neuralmind/bert-large-portuguese-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/neuralmind/bert-large-portuguese-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/neuralmind/bert-large-portuguese-cased/special_tokens_map.json


In [14]:
!rm -rf ./results/

###  Multi - "bert-base-multilingual-cased"

In [21]:
train_model("bert-base-multilingual-cased")

https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpmimrneck


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/f55e7a2ad4f8d0fff2733b3f79777e1e99247f2e4583703e92ce74453af8c235.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
creating metadata file for /root/.cache/huggingface/transformers/f55e7a2ad4f8d0fff2733b3f79777e1e99247f2e4583703e92ce74453af8c235.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpt4a_2d8d


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
creating metadata file for /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidde

...

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 837
  Batch size = 16


{'epoch': 3.0,
 'eval_accuracy': 0.5878136200716846,
 'eval_f1': 0.5579571457945763,
 'eval_loss': 1.0803804397583008,
 'eval_precision': 0.5720279243140933,
 'eval_recall': 0.553404789177985,
 'eval_runtime': 4.5618,
 'eval_samples_per_second': 183.48,
 'eval_steps_per_second': 11.618}

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 838
  Batch size = 16


PredictionOutput(predictions=array([[ 2.0394526 , -1.7366139 ,  2.0997968 ,  0.18916328, -2.4510672 ],
       [ 3.3359878 , -1.5534112 , -0.36108166,  0.33963686, -2.1870446 ],
       [ 2.667591  , -1.8099991 ,  1.8514398 , -0.49706808, -2.4376795 ],
       ...,
       [ 2.8142035 , -1.766973  ,  0.10617247,  1.221561  , -2.9316494 ],
       [ 0.0885511 , -1.8566151 ,  3.6666324 ,  0.13804786, -1.9898982 ],
       [ 2.5178652 , -2.3226469 ,  1.912825  ,  0.7750264 , -3.0698864 ]],
      dtype=float32), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/bert-base-multilingual-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/bert-base-multilingual-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/bert-base-multilingual-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/bert-base-multilingual-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/bert-base-multilingual-cased/special_tokens_map.json


In [22]:
!rm -rf ./results/

###  Multi - "bert-base-multilingual-uncased"

In [23]:
train_model("bert-base-multilingual-uncased")

https://huggingface.co/bert-base-multilingual-uncased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpvzse413u


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-uncased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/1b935b135ddb021a7d836c00f5702b80d11d348fd5c5a42cbd933d8ed1f55be9.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
creating metadata file for /root/.cache/huggingface/transformers/1b935b135ddb021a7d836c00f5702b80d11d348fd5c5a42cbd933d8ed1f55be9.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
https://huggingface.co/bert-base-multilingual-uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp1rd0cn23


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-uncased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/af4e101d208f361f141144dca21e9c4148aaf0e85441c2e335743d10829c6cad.d63adade93e44e64bedd306ec82ffd33eedabaf0ff08aabe581acaa48616a508
creating metadata file for /root/.cache/huggingface/transformers/af4e101d208f361f141144dca21e9c4148aaf0e85441c2e335743d10829c6cad.d63adade93e44e64bedd306ec82ffd33eedabaf0ff08aabe581acaa48616a508
loading configuration file https://huggingface.co/bert-base-multilingual-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/af4e101d208f361f141144dca21e9c4148aaf0e85441c2e335743d10829c6cad.d63adade93e44e64bedd306ec82ffd33eedabaf0ff08aabe581acaa48616a508
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  

...

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 837
  Batch size = 16


{'epoch': 3.0,
 'eval_accuracy': 0.5806451612903226,
 'eval_f1': 0.5497067702204519,
 'eval_loss': 1.0696624517440796,
 'eval_precision': 0.5616965164783988,
 'eval_recall': 0.5472448589974364,
 'eval_runtime': 4.5805,
 'eval_samples_per_second': 182.73,
 'eval_steps_per_second': 11.571}

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 838
  Batch size = 16


PredictionOutput(predictions=array([[ 1.0068008 ,  1.3165907 , -2.2574716 , -0.64288455,  0.10628735],
       [ 3.5043476 , -1.1374558 , -0.4072238 ,  0.35866255, -2.3819885 ],
       [ 2.4198575 , -1.5340352 ,  2.059636  ,  0.02742954, -2.9252157 ],
       ...,
       [ 1.9868321 , -2.4830842 ,  2.1156018 ,  1.3036941 , -2.6271167 ],
       [ 0.48403916, -1.5527145 ,  3.8177838 , -0.11367209, -2.0803223 ],
       [ 2.7942872 , -1.371459  ,  1.0717577 ,  1.1348625 , -3.3870175 ]],
      dtype=float32), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/bert-base-multilingual-uncased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/bert-base-multilingual-uncased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/bert-base-multilingual-uncased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/bert-base-multilingual-uncased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/bert-base-multilingual-uncased/special_tokens_map.json


In [24]:
!rm -rf ./results/